In [1]:
import random
import re
from ast import literal_eval
import itertools

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet_len = len(alphabet)

In [36]:
# for shift
input_shift = 'input.txt'
output_shift = 'output.txt'
decoded_shift = 'decoded.txt'
shift = 3

# for shuffle
input_shuffle = 'input_shuffle.txt'
output_shuffle = 'output_shuffle.txt'
decoded_shuffle = 'decoded_shuffle.txt'
key = '[[2, 1, 0], [0, 1, 2]]'

# for gamma
input_gamma = 'input_gamma.txt'
output_gamma = 'output_gamma.txt'
decoded_gamma = 'decoded_gamma.txt'
key_for_gamma = 'hf'

# for moving probable word cryptanalysis
length_shuffle = 3
length_gamma = 2
number = 2
probable_word = 'lion'
encryption_type = 'gamma'

In [17]:
func_shift = 'decode'
func_shuffle = 'decode'
func_gamma = 'decode'

# Шифрование методом сдвига

In [18]:
def encode_shift(c, shift):
    if c not in alphabet:
        return c
    index = (alphabet.index(c) + shift) % alphabet_len
    return alphabet[index]

In [19]:
def decode_shift(c, shift):
    if c not in alphabet:
        return c
    index = (alphabet.index(c) - shift) % alphabet_len
    return alphabet[index]

In [20]:
if func_shift == 'encode':
    with open(input_shift, "r") as fp_input:
        with open(output_shift, "w") as fp_output:
            while True:
                c = fp_input.read(1).lower()
                #print(c)
                if not c:
                    break
                fp_output.write(encode_shift(c, shift))
elif func_shift == 'decode':
    with open(output_shift, "r") as fp_input:
        with open(decoded_shift, "w") as fp_output:
            while True:
                c = fp_input.read(1).lower()
                #print(c)
                if not c:
                    break
                fp_output.write(decode_shift(c, shift))

# Шифрование методом перестановки

In [21]:
def encode_shuffle(cs, key):
    new_buffer = [None] * len(cs)
    cur_len = 0
    for subkey in key:
        for ind, val in enumerate(subkey):
            new_buffer[cur_len + val] = cs[cur_len + ind]
        cur_len += len(subkey)
    return "".join(new_buffer)

In [22]:
def decode_shuffle(cs, key):
    new_buffer = [None] * len(cs)
    cur_len = 0
    for subkey in key:
        for ind, val in enumerate(subkey):
            new_buffer[cur_len + int(ind)] = cs[cur_len + int(val)]
        cur_len += len(subkey)
    return "".join(new_buffer)

In [23]:
key = literal_eval(key)
total_len = sum(map(len, key))
if func_shuffle == 'encode':
    with open(input_shuffle, "r") as fp_input:
        with open(output_shuffle, "w") as fp_output:
            continue_cycle = True
            buffer = []
            while continue_cycle:
                c = fp_input.read(1).lower()
                c = re.sub(r"\s", " ", c)
                if not c:
                    if len(buffer) == 0:
                        break
                    continue_cycle = False
                    while len(buffer) < total_len:
                        buffer.append(random.choice(alphabet))
                else:
                    buffer.append(c)

                if len(buffer) == total_len:
                    fp_output.write(encode_shuffle(buffer, key))
                    buffer = []

elif func_shuffle == 'decode': 
    with open(output_shuffle, "r") as fp_input:
        with open(decoded_shuffle, "w") as fp_output:
            continue_cycle = True
            buffer = []
            while continue_cycle:
                c = fp_input.read(1).lower()
                c = re.sub(r"\s", " ", c)
                if not c:
                    if len(buffer) == 0:
                        break
                    continue_cycle = False
                    while len(buffer) < total_len:
                        buffer.append(random.choice(alphabet))
                else:
                    buffer.append(c)
                if len(buffer) == total_len:
                    fp_output.write(decode_shuffle(buffer, key))
                    buffer = []

# Шифрование методом гаммирования

In [24]:
def encode_gamma(c, key_for_gamma, pos):
    if c not in alphabet:
        return c
    index = (alphabet.index(c) + alphabet.index(key_for_gamma[pos % len(key_for_gamma)]) ) % alphabet_len
    return alphabet[index]

In [25]:
def decode_gamma(c, key_for_gamma, pos):
    if c not in alphabet:
        return c
    index = (alphabet.index(c) - alphabet.index(key_for_gamma[pos % len(key_for_gamma)]) ) % alphabet_len
    return alphabet[index]

In [26]:
if func_gamma == 'encode':
    with open(input_gamma, "r") as fp_input:
        with open(output_gamma, "w") as fp_output:
            pos = 0
            while True:
                c = fp_input.read(1).lower()
                if not c:
                    break
                fp_output.write(encode_gamma(c, key_for_gamma, pos))
                pos += 1

elif func_gamma == 'decode': 
    with open(output_gamma, "r") as fp_input:
        with open(decoded_gamma, "w") as fp_output:
            pos = 0
            while True:
                c = fp_input.read(1).lower()
                if not c:
                    break
                fp_output.write(decode_gamma(c, key_for_gamma, pos))
                pos += 1


# Метод протяжки вероятного слова

In [27]:
def params_for_shift():
    for shift in range(alphabet_len):
        yield shift

def params_for_shuffle(len_, num):
    if num == 1:
        for state in itertools.permutations(range(len_)):
            yield [state]
    else:
        for first_state in itertools.permutations(range(len_)):
            for next_states in params_for_shuffle(len_, num - 1):
                yield [first_state] + next_states

def params_for_gamma(key_len):
    if key_len == 1:
        for c in alphabet:
            yield c
    else:
        for first_char in alphabet:
            for next_chars in params_for_gamma(key_len - 1):
                yield first_char + next_chars

In [28]:
def decode_shift(input_text, shift):
    decoded_text = ""
    for c in input_text:
        if c not in alphabet:
            decoded_text += c
        else:
            decoded_text += alphabet[(alphabet.index(c) - shift) % alphabet_len]
    return decoded_text

def decode_shuffle(input_text, key):
    total_len = sum(map(len, key))
    input_text = iter(input_text)
    decoded_text = ""
    continue_cycle = True
    buffer = []
    while continue_cycle:
        try:
            c = next(input_text).lower()
        except StopIteration:
            c = ""
        c = re.sub(r"\s", " ", c)
        if not c:
            if len(buffer) == 0:
                break
            continue_cycle = False
            while len(buffer) < total_len:
                buffer.append(random.choice(alphabet))
        else:
            buffer.append(c)
        if len(buffer) == total_len:
            new_buffer = [None] * len(buffer)
            cur_len = 0
            for subkey in key:
                for ind, val in enumerate(subkey):
                    new_buffer[cur_len + ind] = buffer[cur_len + val]
                cur_len += len(subkey)
            decoded_text += "".join(new_buffer)
            buffer = []
    return decoded_text

def decode_gamma(input_text, key):
    decoded_text = ""
    for pos, c in enumerate(input_text):
        if c not in alphabet:
            decoded_text += c
        else:
            decoded_text += alphabet[
                (alphabet.index(c) - alphabet.index(key[pos % len(key)])) % alphabet_len]
    return decoded_text

In [37]:
i = 0
list_of_keys = []

if encryption_type == "shift":
    iterator = params_for_shift()
    decoder_func = decode_shift
    input_file = output_shift

if encryption_type== "shuffle":
    iterator = params_for_shuffle(length_shuffle, number)
    decoder_func = decode_shuffle
    input_file = output_shuffle

if encryption_type == "gamma":
    iterator = params_for_gamma(length_gamma)
    decoder_func = decode_gamma
    input_file = output_gamma

with open(input_file, "r") as fp_input:
    input_text = fp_input.read()
for key in iterator:
    decoded_text = decoder_func(input_text, key)
    if probable_word in decoded_text:
        list_of_keys.append(key)
        print(f"{i} ключ: {key}")
        i += 1

0 ключ: hf


In [38]:
input_key_num = input("Выберите номер ключа, чтобы вывести на экран кусочек расшифрованного текста (например: 0): \n")
print(decoder_func(input_text, list_of_keys[int(input_key_num)]))

Выберите номер ключа, чтобы вывести на экран кусочек расшифрованного текста (например: 0): 
0
a lion was once sleeping in the jungle when a mouse started running up and down his body just for fun. this disturbed the lion’s sleep, and he woke up quite angry. he was about to eat the mouse when the mouse desperately requested the lion to set him free. i promise you, i will be of great help to you someday if you save me. the lion laughed at the mouse’s confidence and let him go. one day, a few hunters came into the forest and took the lion with them. they tied him up against a tree. the lion was struggling to get out and started to whimper. soon, the mouse walked past and noticed the lion in trouble. quickly, he ran and gnawed on the ropes to set the lion free. both of them sped off into the jungle.
